
# Modelo de Classificação de escolas baseada em dados do INEP (2014-2023)

## Bibliotecas

In [31]:
import pandas as pd
import wbdata
import datetime
import pycountry
import geopandas


## Importação dos Dados

In [6]:
# Download latest version
path = "./dataset/master.csv"

# Carregar o arquivo CSV no DataFrame
df = pd.read_csv(path)


## Análise Exploratória de Dados

In [7]:
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [ ]:
# Remover coluna country-year por motivos de redundância
df = df.drop(columns=["country-year"])

In [12]:
antes = len(df)

# Removendo Duplicatas
df.drop_duplicates(ignore_index=True,inplace=True)

depois = len(df)

print(f"Linhas duplicadas existentes e removidas = {depois - antes}")

Linhas duplicadas existentes e removidas = 0


- Informações sobre Dataframe

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             27820 non-null  object 
 1   year                27820 non-null  int64  
 2   sex                 27820 non-null  object 
 3   age                 27820 non-null  object 
 4   suicides_no         27820 non-null  int64  
 5   population          27820 non-null  int64  
 6   suicides/100k pop   27820 non-null  float64
 7   HDI for year        8364 non-null   float64
 8    gdp_for_year ($)   27820 non-null  object 
 9   gdp_per_capita ($)  27820 non-null  int64  
 10  generation          27820 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 2.3+ MB


- Estatística Básica

In [14]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
year,27820.0,2.001258e+03,8.469055e+00,1985.000,1995.000,2002.000,2008.000,2.016000e+03
suicides_no,27820.0,2.425744e+02,9.020479e+02,0.000,3.000,25.000,131.000,2.233800e+04
population,27820.0,1.844794e+06,3.911779e+06,278.000,97498.500,430150.000,1486143.250,4.380521e+07
suicides/100k pop,27820.0,1.281610e+01,1.896151e+01,0.000,0.920,5.990,16.620,2.249700e+02
HDI for year,8364.0,7.766011e-01,9.336671e-02,0.483,0.713,0.779,0.855,9.440000e-01
gdp_per_capita ($),27820.0,1.686646e+04,1.888758e+04,251.000,3447.000,9372.000,24874.000,1.263520e+05


- Contando Nulos ou NaN

In [25]:
len(df)

27820

In [24]:
df.isnull().sum()

country                   0
year                      0
sex                       0
age                       0
suicides_no               0
population                0
suicides/100k pop         0
HDI for year          19456
 gdp_for_year ($)         0
gdp_per_capita ($)        0
generation                0
dtype: int64

In [18]:
df.head(20)

,country,year,sex,age,suicides_no,population,suicides/100k pop,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,NaN,"2,156,624,900",796,Boomers
5,Albania,1987,female,75+ years,1,35600,2.81,NaN,"2,156,624,900",796,G.I. Generation
6,Albania,1987,female,35-54 years,6,278800,2.15,NaN,"2,156,624,900",796,Silent
7,Albania,1987,female,25-34 years,4,257200,1.56,NaN,"2,156,624,900",796,Boomers
8,Albania,1987,male,55-74 years,1,137500,0.73,NaN,"2,156,624,900",796,G.I. Generation
9,Albania,1987,female,5-14 years,0,311000,0.00,NaN,"2,156,624,900",796,Generation X


- Preencher os dados nulos com informações do World Bank

In [ ]:
# Função para mapear o nome do país para seu código ISO (2 letras)
def get_country_code(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        return country.alpha_2  # Código ISO de 2 letras
    except AttributeError:
        return None  # Retorna None se o país não for encontrado

# Função para buscar PIB
def get_gdp_data(country_code, year):
    indicator = {'NY.GDP.MKTP.CD': 'GDP'}
    start_date = datetime.datetime(year, 1, 1)
    end_date = datetime.datetime(year, 12, 31)
    
    try:
        gdp_data = wbdata.get_dataframe(indicator, country=country_code, data_date=(start_date, end_date))
        return gdp_data['GDP'][0]
    except:
        return None

# Exemplo: Preenchendo com PIB de uma API
for index, row in df[df['HDI for year'].isnull()].iterrows():
    country = row['country']
    year = row['year']
    country_code = get_country_code(country)  # Aqui você precisaria de uma função para buscar o código ISO
    gdp_value = get_gdp_data(country_code, year)
    
    if gdp_value:
        df.at[index, 'HDI for year'] = gdp_value  # Preenche o valor nulo com o PIB
